In [1]:
# All  import statements needed for the notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.metrics import *

In [2]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
data = pd.read_csv('/content/drive/MyDrive/balancedhalf_data.csv')

data.drop(data.columns[0] , inplace=True , axis=1)

unknown = ['SSH-Patator','DoS slowloris','DoS Slowhttptest','Bot','Infiltration','Heartbleed']
att = data.loc[(data['Label'].isin(unknown))]

data.drop(att.index,axis=0 , inplace=True, errors='ignore')

In [5]:
## PreProcessing

# importing required libraries for normalizing data
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
# selecting numeric attributes columns from data
numeric_col = data.select_dtypes(include='number').columns

# using standard scaler for normalizing
std_scaler = MinMaxScaler()
def normalization(df,att,col):
  for i in col:
    arr = df[i]
    arr = np.array(arr)
    x = np.array(att[i])
    df[i] = std_scaler.fit_transform(arr.reshape(len(arr),1))
    #To use the same scaler which was used in preprocessing the train data 
    att[i] = std_scaler.transform(x.reshape(len(x),1))
  return df,att
# calling the normalization() function
data , att = normalization(data.copy(),att.copy(),numeric_col)

data.shape , att.shape

att.Label = 'unknown'

X = att.drop('Label' , axis=1)
X = X.to_numpy().reshape(-1, 83,1)

y = att.Label

X_train = data.drop('Label' , axis=1)
X_train = X_train.to_numpy().reshape(-1, 83,1)

y_train = data.Label

In [6]:
from sklearn.model_selection import train_test_split
X_train_set, X_test_set, y_train_set, y_test_set = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

In [7]:
# **Load Model and Predict**
from tensorflow import keras
model = keras.models.load_model('/content/drive/MyDrive/models/model_split_softmax_cnn_model.hdf5')

In [8]:
Y_train_predicted = model.predict(X_train_set)

22859/22859 [==============================] - 61s 3ms/step


In [9]:
LABELS = ['BENIGN','DDoS','DoS Hulk','DoS GoldenEye','PortScan','FTP-Patator']

In [10]:
y_prediction_index = np.argmax(Y_train_predicted, axis = 1)

In [11]:
df = pd.DataFrame(data=Y_train_predicted,columns=LABELS)

In [12]:
df.head(1)

,BENIGN,DDoS,DoS Hulk,DoS GoldenEye,PortScan,FTP-Patator
0,0.0,0.0,0.0,0.0,0.0,1.0


In [13]:
pd.Series(y_prediction_index).value_counts()

0    371895
3    155203
5    106613
1     85603
2      6837
4      5306
dtype: int64

In [14]:
labeltoindex = {
    'BENIGN':0,'DDoS':1,'DoS Hulk':2,'DoS GoldenEye':3,'PortScan':4,'FTP-Patator':5
}
TRUELY_PREDICTED = np.vectorize(labeltoindex.get)(y_train_set) == y_prediction_index

In [15]:
df['INDEX_WIN'] = np.where( TRUELY_PREDICTED == True , y_prediction_index, '')

In [16]:
df.head()

,BENIGN,DDoS,DoS Hulk,DoS GoldenEye,PortScan,FTP-Patator,INDEX_WIN
0,0.000000e+00,0.0,0.000000e+00,0.0,0.0,1.0,
1,1.000000e+00,0.0,0.000000e+00,0.0,0.0,0.0,0
2,0.000000e+00,0.0,0.000000e+00,1.0,0.0,0.0,
3,2.404753e-26,0.0,4.052720e-36,1.0,0.0,0.0,
4,1.000000e+00,0.0,0.000000e+00,0.0,0.0,0.0,0


In [17]:
y_prediction_index[2]

3

In [18]:
df['INDEX_WIN'] = y_prediction_index

In [19]:
df['INDEX_WIN'].value_counts()

0    371895
3    155203
5    106613
1     85603
2      6837
4      5306
Name: INDEX_WIN, dtype: int64

In [20]:
df = df[df['INDEX_WIN'] != '']

In [21]:
df.shape

(731457, 7)

In [22]:
df.head()

,BENIGN,DDoS,DoS Hulk,DoS GoldenEye,PortScan,FTP-Patator,INDEX_WIN
0,0.000000e+00,0.0,0.000000e+00,0.0,0.0,1.0,5
1,1.000000e+00,0.0,0.000000e+00,0.0,0.0,0.0,0
2,0.000000e+00,0.0,0.000000e+00,1.0,0.0,0.0,3
3,2.404753e-26,0.0,4.052720e-36,1.0,0.0,0.0,3
4,1.000000e+00,0.0,0.000000e+00,0.0,0.0,0.0,0


In [23]:
df.dtypes

BENIGN           float32
DDoS             float32
DoS Hulk         float32
DoS GoldenEye    float32
PortScan         float32
FTP-Patator      float32
INDEX_WIN          int64
dtype: object

In [24]:
df.INDEX_WIN = df['INDEX_WIN'].astype('float32')

In [25]:
def to_do(df):
  ind = df['INDEX_WIN'].to_numpy()[0]
  LABELS = ['BENIGN','DDoS','DoS Hulk','DoS GoldenEye','PortScan','FTP-Patator']
  label = LABELS[ind] 
  import numpy as np
  threshold =  np.amin(df[label])
  #print(label , threshold)
  threshold_values[label] = threshold
  #return threshold

In [43]:
arr = df.iloc[0].to_numpy()
max(arr[:6])
result = np.where(arr[:6] == max(arr[:6]))
result
np.argmax(arr[:6])
arr[:6]

array([0., 0., 0., 0., 0., 1.], dtype=float32)

In [44]:
threshold_values = {'BENIGN':1.0,'DDoS':1.0,'DoS Hulk':1.0,'DoS GoldenEye':1.0,'PortScan':1.0,'FTP-Patator':1.0}
LABELS = ['BENIGN','DDoS','DoS Hulk','DoS GoldenEye','PortScan','FTP-Patator']
for i in range(len(df)):
  arr = df.iloc[i].to_numpy()
  index = np.argmax(arr[:6])
  value = max(arr)
  if threshold_values[LABELS[index]] > value:
    threshold_values[LABELS[index]] = value

In [45]:
threshold_values

{'BENIGN': 0.37627712,
 'DDoS': 1.0,
 'DoS Hulk': 1.0,
 'DoS GoldenEye': 1.0,
 'PortScan': 1.0,
 'FTP-Patator': 1.0}

In [27]:
df.groupby(['INDEX_WIN'], group_keys=True).apply(to_do)
threshold_values

KeyError: ignored

In [ ]:
threshold_values

## Evaluation

In [ ]:
df[df['BENIGN'] == 0.009408989]